In [52]:
from google.colab import drive
drive.mount('/content/drive/')

PATH = '/content/drive/My Drive/Colab Notebooks/SOYO/Election/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install -q pymysql

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.4).
default-libmysqlclient-dev set to manually installed.
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
     |████████████████████████████████| 43 kB 1.7 MB/s 


In [3]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from datetime import timedelta

import pymysql

In [4]:
!pip install -q konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 61.2 MB/s 


In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
host = '****.*****.ap-northeast-2.rds.amazonaws.com'
port = 3306
username = '***'
password = '****'
db = '****'

conn = pymysql.connect(host=host, port=port, user=username, password=password, db=db)

In [17]:
# 유튜브 라이브 댓글
sql_state = 'SELECT * FROM youtube_comment_live'

youtube_df = pd.read_sql_query(sql_state, conn)
conn.close()

In [18]:
# 새로 가져온 영상 source들
print(youtube_df.shape)
youtube_df['youtube_id'].value_counts() # 10: mbc(3차), 11: sbs(3차), 12: kbs(4차), 13: mbc(4차)

(306667, 6)


13    104675
10     96913
12     55659
11     49420
Name: youtube_id, dtype: int64

In [19]:
mbc_3rd = youtube_df.loc[youtube_df['youtube_id'] == 10]
sbs_3rd = youtube_df.loc[youtube_df['youtube_id'] == 11]
kbs_4th = youtube_df.loc[youtube_df['youtube_id'] == 12]
mbc_4th = youtube_df.loc[youtube_df['youtube_id'] == 13]

mbc_3rd.reset_index(drop=True, inplace=True)
sbs_3rd.reset_index(drop=True, inplace=True)
kbs_4th.reset_index(drop=True, inplace=True)
mbc_4th.reset_index(drop=True, inplace=True)

In [26]:
# stopwords 목록 가져오기
from nltk.tokenize import word_tokenize

kor_stopwords = []
with open('/content/drive/My Drive/Colab Notebooks/SOYO/Election/dataset/korean_stopwords.txt', 'r') as f:
  for line in f:
    kor_stopwords.append(line.strip())

In [27]:
def onlyhangul(x):
  pattern = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글 및 띄어쓰기에 해당되지 않는 부분(영어, 기호, 이모티콘 등)
  results = pattern.sub('', str(x))
  # removed_string = pattern.findall(x)
  return results

def remove_stopwords(x):
  result = []
  word_tokens = word_tokenize(str(x))
  for w in word_tokens:
      if w not in kor_stopwords:
          result.append(w)
  
  if len(result) == 0:
    return 0
  return ' '.join(result)

# 유튜브 라이브 댓글 전처리

In [30]:
# 유튜브 댓글 중에 ??, ㅋㅋㅋ, 응원합니다 등의 stopwords 제거
for wd in youtube_df['content'].value_counts()[:17].index.tolist():
  kor_stopwords.append(wd)
print(len(kor_stopwords))

692


In [31]:
mbc_3rd['cleaned'] = mbc_3rd['content'].apply(lambda x: remove_stopwords(onlyhangul(x)))
sbs_3rd['cleaned'] = sbs_3rd['content'].apply(lambda x: remove_stopwords(onlyhangul(x)))
kbs_4th['cleaned'] = kbs_4th['content'].apply(lambda x: remove_stopwords(onlyhangul(x)))
mbc_4th['cleaned'] = mbc_4th['content'].apply(lambda x: remove_stopwords(onlyhangul(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [33]:
mbc_3rd = mbc_3rd.loc[mbc_3rd['cleaned'] != 0]
sbs_3rd = sbs_3rd.loc[sbs_3rd['cleaned'] != 0]
kbs_4th = kbs_4th.loc[kbs_4th['cleaned'] != 0]
mbc_4th = mbc_4th.loc[mbc_4th['cleaned'] != 0]

print(mbc_3rd.shape)
print(sbs_3rd.shape)
print(kbs_4th.shape)
print(mbc_4th.shape)

(79340, 7)
(38268, 7)
(39502, 7)
(82489, 7)


## target_candi가 Null인 댓글 삭제

In [49]:
mbc_3rd = mbc_3rd.loc[~mbc_3rd['target_candi'].isnull()]
sbs_3rd = sbs_3rd.loc[~sbs_3rd['target_candi'].isnull()]
kbs_4th = kbs_4th.loc[~kbs_4th['target_candi'].isnull()]
mbc_4th = mbc_4th.loc[~mbc_4th['target_candi'].isnull()]

## 이재명 대통령, 윤석열 대통령만 있는 댓글 삭제

In [72]:
mbc_3rd = mbc_3rd.loc[mbc_3rd['cleaned'] != '대통령']
sbs_3rd = sbs_3rd.loc[sbs_3rd['cleaned'] != '대통령']
kbs_4th = kbs_4th.loc[kbs_4th['cleaned'] != '대통령']
mbc_4th = mbc_4th.loc[mbc_4th['cleaned'] != '대통령']

mbc_3rd.reset_index(drop=True, inplace=True)
sbs_3rd.reset_index(drop=True, inplace=True)
kbs_4th.reset_index(drop=True, inplace=True)
mbc_4th.reset_index(drop=True, inplace=True)

In [ ]:
# 뉴스 댓글
# sql_state = 'SELECT comment FROM news_comment A JOIN news_list B ON B.title=A.title'

# news_df = pd.read_sql_query(sql_state, conn)
# conn.close()

In [40]:
!pip install -q mxnet
!pip install -q gluonnlp pandas tqdm
!pip install -q sentencepiece
!pip install -q transformers>=4.8.1
!pip install -q torch

     |████████████████████████████████| 47.3 MB 1.4 MB/s 
     |████████████████████████████████| 127 kB 70.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 344 kB 7.5 MB/s 
     |████████████████████████████████| 1.2 MB 8.4 MB/s 


In [41]:
!pip install -q git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 4.5 MB 9.4 MB/s 


In [42]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp

from tqdm import tqdm

In [43]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [44]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [45]:
# GPU 사용 시
device = torch.device("cuda:0")
print(device)

bertmodel, vocab = get_pytorch_kobert_model()

cuda:0
/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# KoBERT 추론

In [46]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [47]:
# 학습 때와 달리 label 파라미터가 필요없으므로 삭제
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, bert_tokenizer, max_len,
                pad, pair):
    transform = nlp.data.BERTSentenceTransform(
        bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

    self.sentences = [transform([i[sent_idx]]) for i in dataset]

  def __getitem__(self, i):
    return self.sentences[i]

  def __len__(self): # valid_length를 위해 필요한 내장 함수
    return len(self.sentences)

In [48]:
class BERTClassifier(nn.Module):
  def __init__(self,
                bert,
                hidden_size=768,
                num_classes=3, # softmax 사용 <- binary일 경우는 2
                dr_rate=None,
                params=None):
    
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate
              
    self.classifier = nn.Linear(hidden_size , num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)
  
  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
    if self.dr_rate:
      out = self.dropout(pooler)
    return self.classifier(out)

model = BERTClassifier(bertmodel, dr_rate=0.5).to(device) # 모델 초기화

* model.pt 자체를 불러오면 에러 발생 -> 이는 새로운 데이터에 따라 모델이 조금이라도 달라지만 문제가 발생한다고 함
* https://discuss.huggingface.co/t/bertencoder-object-has-no-attribute-gradient-checkpointing/11207

In [53]:
# 저장된 모델 불러오기
model.load_state_dict(torch.load(PATH + 'model_state.pt')) 
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [55]:
# setting parameters
max_len = 128

mapping = {0: '중립', 1: '긍정', 2: '부정'}

### mbc 3차

In [74]:
unseen_test = mbc_3rd[['cleaned']]
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, tok, max_len, True, False)

test_input = DataLoader(test_set, batch_size=1, num_workers=2) # colab은 max num works가 2

result_list1 = []

for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)

  max_vals, max_indices = torch.max(out, 1)
  result_list1.append(max_indices) # 중립: 0, 긍정: 1, 부정: 2

100%|██████████| 43081/43081 [08:25<00:00, 85.25it/s]


### sbs 3차

In [75]:
unseen_test = sbs_3rd[['cleaned']]
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, tok, max_len, True, False)

test_input = DataLoader(test_set, batch_size=1, num_workers=2)

result_list2 = []

for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)

  max_vals, max_indices = torch.max(out, 1)
  result_list2.append(max_indices) # 중립: 0, 긍정: 1, 부정: 2

100%|██████████| 20042/20042 [03:51<00:00, 86.74it/s]


### kbs 4차

In [76]:
unseen_test = kbs_4th[['cleaned']]
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, tok, max_len, True, False)

test_input = DataLoader(test_set, batch_size=1, num_workers=2)

result_list3 = []

for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)

  max_vals, max_indices = torch.max(out, 1)
  result_list3.append(max_indices) # 중립: 0, 긍정: 1, 부정: 2

100%|██████████| 19956/19956 [04:38<00:00, 71.60it/s]


### mbc 4차

In [77]:
unseen_test = mbc_4th[['cleaned']]
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, tok, max_len, True, False)

test_input = DataLoader(test_set, batch_size=1, num_workers=2)

result_list4 = []

for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)

  max_vals, max_indices = torch.max(out, 1)
  result_list4.append(max_indices) # 중립: 0, 긍정: 1, 부정: 2

100%|██████████| 43864/43864 [08:20<00:00, 87.64it/s]


In [78]:
# mapping 변환
new_list1 = []
new_list2 = []
new_list3 = []
new_list4 = []
for ele in result_list1:
  new_list1.append(mapping[int(ele)])
for ele in result_list2:
  new_list2.append(mapping[int(ele)])
for ele in result_list3:
  new_list3.append(mapping[int(ele)])
for ele in result_list4:
  new_list4.append(mapping[int(ele)])

In [79]:
youtube_result1 = pd.concat([mbc_3rd, pd.DataFrame(new_list1, columns=['emotion'])], axis=1)
youtube_result2 = pd.concat([sbs_3rd, pd.DataFrame(new_list2, columns=['emotion'])], axis=1)
youtube_result3 = pd.concat([kbs_4th, pd.DataFrame(new_list3, columns=['emotion'])], axis=1)
youtube_result4 = pd.concat([mbc_4th, pd.DataFrame(new_list4, columns=['emotion'])], axis=1)

In [80]:
youtube_result1.to_csv(PATH + 'dataset/mbc_3rd_sentiment.csv', index=False, encoding='utf-8-sig')
youtube_result2.to_csv(PATH + 'dataset/sbs_3rd_sentiment.csv', index=False, encoding='utf-8-sig')
youtube_result3.to_csv(PATH + 'dataset/kbs_4th_sentiment.csv', index=False, encoding='utf-8-sig')
youtube_result4.to_csv(PATH + 'dataset/mbc_4th_sentiment.csv', index=False, encoding='utf-8-sig')